In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI模型花园 - 稳定视频扩散 Img2Vid XT（研究目的）

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_video_diffusion_img2vid_xt.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/model_garden/model_garden_pytorch_stable_video_diffusion_img2vid_xt.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_video_diffusion_img2vid_xt.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
在Vertex AI Workbench中打开
    </a>
    （建议使用预装的HuggingFace/transformer库的Python-3 GPU笔记本）
  </td>
</table>

## 概述

此笔记本演示了在[Colab](https://colab.research.google.com)或[Vertex AI Workbench](https://cloud.google.com/vertex-ai-workbench)上运行[stabilityai/stable-video-diffusion-img2vid-xt](https://huggingface.co/stabilityai/stable-video-diffusion-img2vid-xt)的本地推断。此笔记本还演示了将其部署到Vertex AI以进行批量预测，仅用于研究目的。

### 目标

- 运行图像到视频的本地预测。
- 上传模型并运行图像到视频的批量预测。

### 成本

此教程使用Google Cloud的计费组件：

* Vertex AI
* 云存储

了解[Vertex AI定价](https://cloud.google.com/vertex-ai/pricing)和[云存储定价](https://cloud.google.com/storage/pricing)，并使用[定价计算器](https://cloud.google.com/products/calculator/)根据您的预期使用情况生成成本估算。

## 开始之前

**注意**: Jupyter会将以`!`为前缀的行视为Shell命令，并将以`$`为前缀的Python变量插入这些命令中。

###设置笔记本

只在Colab上运行以下命令，如果您正在使用工作台，请跳过这部分。

In [ ]:
if "google.colab" in str(get_ipython()):
    ! pip3 install --upgrade google-cloud-aiplatform
    from google.colab import auth as google_auth

    google_auth.authenticate_user()

! pip3 install --upgrade pip
! pip3 install torch==2.0.1+cu118  -f https://download.pytorch.org/whl/torch_stable.html
! pip3 install transformers==4.36.2
! pip3 install diffusers==0.25.0
! pip3 install datasets==2.9.0
! pip3 install accelerate==0.25.0

# Install gdown for downloading example training images.
! pip3 install gdown
# Remove wrong cublas version.
! pip3 uninstall nvidia_cublas_cu11 --yes

# Restart the notebook kernel after installs.
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

只有工作台
1. 点击[此链接](https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/model_garden/model_garden_pytorch_stable_video_diffusion_img2vid_xt.ipynb)部署笔记本到Vertex AI工作台实例。
2. 选中`创建一个新笔记本`。
3. 点击`高级选项`。
4. 在**环境**选项卡中，选择`Debian 10`作为**操作系统**，选择`自定义容器`作为**环境**。
5. 将`Docker容器镜像`设置为`us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/transformers-notebook`。
6. 在**机器配置**下，选择1个`T4` GPU，选择`自动为我安装NVIDIA GPU驱动程序`。
7. 点击`创建`来创建Vertex AI工作台实例。

###设置Google Cloud项目

1. [选择或创建一个Google Cloud项目](https://console.cloud.google.com/cloud-resource-manager)。当您第一次创建一个账户时，您将获得300美元的免费信贷用于计算/存储成本。

2. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

3. [启用Vertex AI API和Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

4. [创建一个Cloud Storage存储桶](https://cloud.google.com/storage/docs/creating-buckets)用于存储实验结果。

5. [创建一个服务账号](https://cloud.google.com/iam/docs/service-accounts-create#iam-service-accounts-create-console)并赋予`Vertex AI用户`和`存储对象管理员`角色，以便将优化模型部署到Vertex AI端点。

填写下列变量以便实验环境：

In [ ]:
# Cloud project id.
PROJECT_ID = ""  # @param {type:"string"}

# The region you want to launch jobs in.
REGION = ""  # @param {type:"string"}

# The Cloud Storage bucket for storing experiments output. Fill it without the 'gs://' prefix.
GCS_BUCKET = ""  # @param {type:"string"}

# The service account for deploying fine tuned model.
SERVICE_ACCOUNT = ""  # @param {type:"string"}

初始化Vertex-AI API:

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION, staging_bucket=GCS_BUCKET)

### 定义常量

In [ ]:
# The pre-built serving docker image. It contains serving scripts and models.

SERVE_DOCKER_URI = "us-docker.pkg.dev/vertex-ai/vertex-vision-model-garden-dockers/pytorch-diffusers-serve-opt:20240605_1400_RC00"

### 定义常用函数

In [ ]:
import base64
import glob
import io
import os
from datetime import datetime
from io import BytesIO

import imageio
from diffusers.utils import load_image
from google.cloud import aiplatform, storage
from IPython.display import HTML


def load_and_resize_image(image_url):
    image = load_image(image_url)
    # required by the model
    new_width = round(image.size[0] / 8) * 8
    new_height = round(image.size[1] / 8) * 8
    return image.resize((new_width, new_height))


def create_job_name(prefix):
    user = os.environ.get("USER")
    now = datetime.now().strftime("%Y%m%d_%H%M%S")
    job_name = f"{prefix}-{user}-{now}"
    return job_name


def image_to_base64(image, format="JPEG"):
    buffer = BytesIO()
    image.save(buffer, format=format)
    image_str = base64.b64encode(buffer.getvalue()).decode("utf-8")
    return image_str


def display_video_frames(frames, fps=7, width=500):
    io_obj = io.BytesIO()
    imageio.mimsave(io_obj, frames, format=".mp4", fps=fps)
    video_data = base64.b64encode(io_obj.getvalue()).decode("utf-8")
    html = ""
    html += f"<video controls width={width}>"
    html += f'<source src="data:video/mp4;base64,{video_data}" type="video/mp4">'
    html += "</video>"
    display(HTML(html))


def upload_model(model_id, task):
    model_name = model_id
    serving_env = {
        "MODEL_ID": model_id,
        "TASK": task,
        "DEPLOY_SOURCE": "notebook",
    }
    model = aiplatform.Model.upload(
        display_name=model_name,
        serving_container_image_uri=SERVE_DOCKER_URI,
        serving_container_ports=[7080],
        serving_container_predict_route="/predictions/diffusers_serving",
        serving_container_health_route="/ping",
        serving_container_environment_variables=serving_env,
    )
    return model


def get_bucket_and_blob_name(filepath):
    # The gcs path is of the form gs://<bucket-name>/<blob-name>
    gs_suffix = filepath.split("gs://", 1)[1]
    return tuple(gs_suffix.split("/", 1))


def upload_local_dir_to_gcs(local_dir_path, gcs_dir_path):
    """Uploads files in a local directory to a GCS directory."""
    client = storage.Client()
    bucket_name = gcs_dir_path.split("/")[2]
    bucket = client.get_bucket(bucket_name)
    for local_file in glob.glob(local_dir_path + "/**"):
        if not os.path.isfile(local_file):
            continue
        filename = local_file[1 + len(local_dir_path) :]
        gcs_file_path = os.path.join(gcs_dir_path, filename)
        _, blob_name = get_bucket_and_blob_name(gcs_file_path)
        blob = bucket.blob(blob_name)
        blob.upload_from_filename(local_file)
        print("Copied {} to {}.".format(local_file, gcs_file_path))


def download_gcs_blob_as_json(gcs_file_path):
    """Download GCS blob and convert it to json."""
    client = storage.Client()
    bucket_name, blob_name = get_bucket_and_blob_name(gcs_file_path)
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(blob_name)

    return json.loads(blob.download_as_bytes())

## 图像到视频

In [ ]:
# @title Define model and task
model_id = "stabilityai/stable-video-diffusion-img2vid-xt"
task = "image-to-video"

In [ ]:
# @title Run inferences locally
import torch
from diffusers import StableVideoDiffusionPipeline

pipe = StableVideoDiffusionPipeline.from_pretrained(
    model_id, torch_dtype=torch.float16, variant="fp16"
)
pipe.enable_model_cpu_offload()

# Load the conditioning image
image = load_and_resize_image(
    "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/diffusers/svd/rocket.png"
)

generator = torch.manual_seed(42)
frames_list = pipe([image], decode_chunk_size=8, generator=generator).frames

display_video_frames(frames_list[0], fps=7)

将模型部署以运行预测

将稳定的视频扩散模型部署到“图像到视频”任务中。

一旦部署，您可以使用一批图像运行批处理预测以生成视频。

当部署在一个“NVIDIA_L4” GPU上时，一个请求的平均推断时间约为600秒。

In [ ]:
# @title Define upload model

model = upload_model(model_id=model_id, task=task)

In [ ]:
# @title Prepare input
import json

image = load_and_resize_image(
    "https://images.pexels.com/videos/853848/free-video-853848.jpg"
)
display(image)

instances = [
    {"prompt": "placehoder", "image": image_to_base64(image), "motion_bucket_id": 127}
]

# Convert and write JSON object to file
os.makedirs("bath_prediction_input", exist_ok=True)
with open("bath_prediction_input/input.jsonl", "w") as outfile:
    for item in instances:
        json_str = json.dumps(item)
        outfile.write(json_str)
        outfile.write("\n")

upload_local_dir_to_gcs(
    "bath_prediction_input", f"gs://{GCS_BUCKET}/image_to_video/input"
)

上传模型。对于批量预测，只需要上传模型即可，无需将模型部署到任何端点。

In [ ]:
# @title Run batch prediction

machine_type = "n1-standard-32"
accelerator_type = "NVIDIA_TESLA_P100"
accelerator_count = 2

batch_prediction_job = model.batch_predict(
    instances_format="jsonl",
    job_display_name="batch_predict_image_to_video",
    gcs_source=[f"gs://{GCS_BUCKET}/image_to_video/input/input.jsonl"],
    gcs_destination_prefix=f"gs://{GCS_BUCKET}/image_to_video/output",
    machine_type=machine_type,
    accelerator_type=accelerator_type,
    accelerator_count=accelerator_count,
    starting_replica_count=1,
)

batch_prediction_job.wait()

print(batch_prediction_job.display_name)
print(batch_prediction_job.resource_name)
print(batch_prediction_job.state)

In [ ]:
# @title Display results

gcs_output_file = (
    batch_prediction_job.output_info.gcs_output_directory
    + "/prediction.results-00000-of-00001"
)
prediction_result = download_gcs_blob_as_json(gcs_output_file)


html = ""
html += "<video controls width=500>"
html += f'<source src="data:video/mp4;base64,{prediction_result["prediction"]}" type="video/mp4">'
html += "</video>"
HTML(html)

In [ ]:
# @title Delete model.
model.delete()